In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from baconshor import *

In [2]:
import sys
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds, OptimizeResult
import matplotlib.pyplot as plt
import csv
import time
from typing import List, Union
def mle_decoder_bs(M : int, I : List[List[int]], C : List[int], p : Union[List[float], float]) -> OptimizeResult:
    """
    Solves the MLE decoder problem using a MILP solver.

    Parameters:
        - M: Number of binary variables E_j.
        - I: List of lists. I[i] is the set of errors E_j which flip the check C_i
        - C: Constraint values C_i.
        - p: List of probabilities p_j for each E_j.

    Returns:
        - result: The result object from scipy.optimize.milp.
        
    """
    # Validate input data
    N = len(I)
    assert len(C) == N, "Length of C must be N"

    #can remove
    if np.isscalar(p):
        p = np.full(M, p)
    # elif type(p) is List:
    #     assert len(p) == M, "Length of p must be M"
    # else:
    #     raise ValueError('p must be a float or a list of floats')

    # Idea: The decision variable x contains both E_j and K_i values.
    # i.e. x = [E_1, ..., E_M, K_1, ..., K_N]
    # The scipy milp solver minimizes <c, x> where c is the objective function coefficients. 
    # We can ignore the constant log(1-p_j) in the objective function.

    # Construct the objective function coefficients, by default set the K_i coefficients to zero
    # Note the minus sign for the E_i coeffs because the solver minimizes the objective function
    c = np.array([-np.log(p_j) + np.log(1 - p_j) for p_j in p] + [0]*N) #remove since assuming all p are same

    # Construct equality constraints matrix (constraints are Ax = b)
    A_eq = np.zeros((N, M + N)) # N constraints, (M + N) variables
    b_eq = []
    for i in range(N):

        # Constraint for the check, C_i is: \sum_{Ej in I_i} E_j - 2K_i = (1 - C_i)/2
        # where I_i is the set of errors [...,E_j,...] that flip the check C_i
        
        # Firstly, set the matrix elements to pick out the appropriate E_j values
        for j in I[i]:
            if j >= M:
                raise ValueError(f"E_j index {j} exceeds M-1")
            A_eq[i, j] = 1

        # Next, set matrix elements to pick out the corresponding K_i
        A_eq[i, M + i] = -2

        # Lastly, compute the right-hand side (1 - C_i)/2
        b_eq.append((1 - C[i]) / 2)

    # Define constraints (equality constraints)
    constraints = LinearConstraint(A=A_eq, lb=b_eq, ub=b_eq)

    # Bounds: E_j: binary (0-1), K_i: integer >=0
    bounds = Bounds(lb = [0]*M + [0]*N,
                    ub = [1]*M + [np.inf]*N)

    # Integrality: Both E_j and K_i are integers within bounds (i.e. 1)
    integrality = [1]*(M + N)  

    # Solve the MILP
    res = milp(c           = c,
               constraints = constraints,
               integrality = integrality,
               bounds      = bounds)
    
    return M, N, res

def Print_result(M,N,result):
    guess = []
    if result.success:
        print("Optimization successful.")
        print(">> Maximum objective value:", -result.fun)  # Convert back to maximization
        print(">> Optimal E_j values:")
        for j in range(M):
            print(f"\t E_{j+1} = {round(result.x[j])}")
            guess.append(round(result.x[j]))
        print(">> Optimal K_i values:")
        for i in range(N):
            print(f"\t K_{i+1} = {int(result.x[M + i])}")
    else:
        print("No optimal solution found.")
        print("Status:", result.message)
        print("Details:", result.get("message", "No additional information"))

    return guess

def print_result(M,N,result):
    guess = []
    if result.success:
        # print("Optimization successful.")
        # print(">> Maximum objective value:", -result.fun)  # Convert back to maximization
        # print(">> Optimal E_j values:")
        # for j in range(M):
        #     # print(f"\t E_{j+1} = {round(result.x[j])}")
        #     guess.append(round(result.x[j]))
        return np.round(result.x[:M]).astype(int)
        # print(">> Optimal K_i values:")
        # for i in range(N):
        #     print(f"\t K_{i+1} = {int(result.x[M + i])}")
    else:
        print("No optimal solution found.")
        # print("Status:", result.message)
        # print("Details:", result.get("message", "No additional information"))


In [13]:
M = 3
grid = create_grid(3)# no error
add_y_error(grid,[2, 8])
Print(grid)
I= construct_stabilizers_scipy_I(M)
C = construct_stabilizers_scipy_C(I, grid)

p = 0.1


guess = print_result(*mle_decoder_bs(M**2,I,C,p))
print("Solver's guess:")
Print(solver_to_grid_scipy(M,guess))
print("Was it correct?:")
solver_accuracy(3,grid,solver_to_grid_scipy(M,guess))

0 1 0 
0 0 0 
0 1 0 
Solver's guess:
1 0 0 
0 0 0 
1 0 0 
Was it correct?:


True

In [38]:
M = 3
grid = create_grid(3)# no error
add_y_error(grid,[3,6,9])
Print(grid)
I= construct_stabilizers_scipy_I(M)
C = construct_stabilizers_scipy_C(I, grid)

p = 0.1


guess = print_result(*mle_decoder_bs(M**2,I,C,p))
print("Solver's guess:")
Print(solver_to_grid_scipy(M,guess))
print("Was it correct?:")
solver_accuracy(3,grid,solver_to_grid_scipy(M,guess))

0 0 1 
0 0 1 
0 0 1 
Solver's guess:
0 0 0 
1 1 0 
0 0 0 
Was it correct?:


False

In [3]:
lw1d3 = list(combinations(list(range(1, (3**2)+1)),1))
listw1d3 =[]
for l in lw1d3:
    listw1d3.append(list(l))

lw2d3 = list(combinations(list(range(1, (3**2)+1)),2))
listw2d3 =[]
for l in lw2d3:
    listw2d3.append(list(l))

lw3d3 = list(combinations(list(range(1, (3**2)+1)),3))
listw3d3 =[]
for l in lw3d3:
    listw3d3.append(list(l))

In [17]:
#prints the ones that don't work for d = 3, w=1
M = 3
for l in listw1d3:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.1
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if not solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")
    

In [15]:
#prints the ones that don't work for d = 3, w = 2
M = 3

count = 0
for l in listw2d3:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.2
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if not solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        count+=1
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")
print("Total #: ")
print(len(listw2d3))
print("# failed:")
print(count)
    

Error Grid:
1 0 0 
0 1 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 0 1 
---------------
Error Grid:
1 0 0 
0 0 1 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 1 0 
---------------
Error Grid:
1 0 0 
0 0 0 
0 1 0 
Solver's guess:
0 0 0 
0 0 1 
0 0 0 
---------------
Error Grid:
1 0 0 
0 0 0 
0 0 1 
Solver's guess:
0 0 0 
0 1 0 
0 0 0 
---------------
Error Grid:
0 1 0 
1 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 0 1 
---------------
Error Grid:
0 1 0 
0 0 1 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
1 0 0 
---------------
Error Grid:
0 1 0 
0 0 0 
1 0 0 
Solver's guess:
0 0 0 
0 0 1 
0 0 0 
---------------
Error Grid:
0 1 0 
0 0 0 
0 0 1 
Solver's guess:
0 0 0 
1 0 0 
0 0 0 
---------------
Error Grid:
0 0 1 
1 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 1 0 
---------------
Error Grid:
0 0 1 
0 1 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
1 0 0 
---------------
Error Grid:
0 0 1 
0 0 0 
1 0 0 
Solver's guess:
0 0 0 
0 1 0 
0 0 0 
---------------
Error Grid:
0 0 1 
0 0 0 
0 1 0 
Solver's guess:
0 0 0

In [ ]:
#prints the ones that do work for d = 3, w = 2
M = 3

count = 0
for l in listw2d3:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.2
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        count+=1
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")

    

Error Grid:
1 1 0 
0 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
1 1 0 
---------------
Error Grid:
1 0 1 
0 0 0 
0 0 0 
Solver's guess:
1 0 1 
0 0 0 
0 0 0 
---------------
Error Grid:
1 0 0 
1 0 0 
0 0 0 
Solver's guess:
1 0 0 
1 0 0 
0 0 0 
---------------
Error Grid:
1 0 0 
0 0 0 
1 0 0 
Solver's guess:
1 0 0 
0 0 0 
1 0 0 
---------------
Error Grid:
0 1 1 
0 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 1 1 
---------------
Error Grid:
0 1 0 
0 1 0 
0 0 0 
Solver's guess:
1 0 0 
1 0 0 
0 0 0 
---------------
Error Grid:
0 1 0 
0 0 0 
0 1 0 
Solver's guess:
1 0 0 
0 0 0 
1 0 0 
---------------
Error Grid:
0 0 1 
0 0 1 
0 0 0 
Solver's guess:
1 0 0 
1 0 0 
0 0 0 
---------------
Error Grid:
0 0 1 
0 0 0 
0 0 1 
Solver's guess:
1 0 0 
0 0 0 
1 0 0 
---------------
Error Grid:
0 0 0 
1 1 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 0 
1 1 0 
---------------
Error Grid:
0 0 0 
1 0 1 
0 0 0 
Solver's guess:
1 0 1 
0 0 0 
0 0 0 
---------------
Error Grid:
0 0 0 
1 0 0 
1 0 0 
Solver's guess:
0 0 0

In [21]:
#prints the ones that do work for d = 3, w = 3
M = 3

for l in listw3d3:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.3
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    
    print("Error Grid:")
    Print(grid)
    print("Solver's guess:")
    Print(solver_to_grid_scipy(M,guess))
    print(solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)))
    print("---------------")


Error Grid:
1 1 1 
0 0 0 
0 0 0 
Solver's guess:
0 0 0 
1 0 0 
1 0 0 
False
---------------
Error Grid:
1 1 0 
1 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 1 0 
0 0 0 
True
---------------
Error Grid:
1 1 0 
0 1 0 
0 0 0 
Solver's guess:
0 0 0 
1 0 0 
0 0 0 
True
---------------
Error Grid:
1 1 0 
0 0 1 
0 0 0 
Solver's guess:
1 0 0 
0 0 0 
1 0 0 
False
---------------
Error Grid:
1 1 0 
0 0 0 
1 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 1 0 
True
---------------
Error Grid:
1 1 0 
0 0 0 
0 1 0 
Solver's guess:
0 0 0 
0 0 0 
1 0 0 
True
---------------
Error Grid:
1 1 0 
0 0 0 
0 0 1 
Solver's guess:
1 0 0 
1 0 0 
0 0 0 
False
---------------
Error Grid:
1 0 1 
1 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 1 
0 0 0 
True
---------------
Error Grid:
1 0 1 
0 1 0 
0 0 0 
Solver's guess:
1 0 0 
0 0 0 
1 0 0 
False
---------------
Error Grid:
1 0 1 
0 0 1 
0 0 0 
Solver's guess:
0 0 0 
1 0 0 
0 0 0 
True
---------------
Error Grid:
1 0 1 
0 0 0 
1 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 0 1 
True
-----------

In [ ]:
#prints the ones that do work for d = 3, w = 3
M = 3

count = 0
for l in listw3d3:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.2
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        count+=1
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")
print("Total #: ")
total = len(listw3d3)
print(total)
print("# failed:")
print(total - count)

Error Grid:
1 1 0 
1 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 1 0 
0 0 0 
---------------
Error Grid:
1 1 0 
0 1 0 
0 0 0 
Solver's guess:
0 0 0 
1 0 0 
0 0 0 
---------------
Error Grid:
1 1 0 
0 0 0 
1 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 1 0 
---------------
Error Grid:
1 1 0 
0 0 0 
0 1 0 
Solver's guess:
0 0 0 
0 0 0 
1 0 0 
---------------
Error Grid:
1 0 1 
1 0 0 
0 0 0 
Solver's guess:
0 0 0 
0 0 1 
0 0 0 
---------------
Error Grid:
1 0 1 
0 0 1 
0 0 0 
Solver's guess:
0 0 0 
1 0 0 
0 0 0 
---------------
Error Grid:
1 0 1 
0 0 0 
1 0 0 
Solver's guess:
0 0 0 
0 0 0 
0 0 1 
---------------
Error Grid:
1 0 1 
0 0 0 
0 0 1 
Solver's guess:
0 0 0 
0 0 0 
1 0 0 
---------------
Error Grid:
1 0 0 
1 1 0 
0 0 0 
Solver's guess:
0 1 0 
0 0 0 
0 0 0 
---------------
Error Grid:
1 0 0 
1 0 1 
0 0 0 
Solver's guess:
0 0 1 
0 0 0 
0 0 0 
---------------
Error Grid:
1 0 0 
0 0 0 
1 1 0 
Solver's guess:
0 1 0 
0 0 0 
0 0 0 
---------------
Error Grid:
1 0 0 
0 0 0 
1 0 1 
Solver's guess:
0 0 1

In [24]:
lw1d5 = list(combinations(list(range(1, (5**2)+1)),1))
listw1d5 =[]
for l in lw1d5:
    listw1d5.append(list(l))

lw2d5 = list(combinations(list(range(1, (5**2)+1)),2))
listw2d5 =[]
for l in lw2d5:
    listw2d5.append(list(l))

lw3d5 = list(combinations(list(range(1, (5**2)+1)),3))
listw3d5 =[]
for l in lw3d5:
    listw3d5.append(list(l))

In [26]:
#prints the ones that don't work for d = 5, w = 1
M = 5
count = 0
for l in listw1d5:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.04
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if not solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        count+=1
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")
print("Total #: ")
total = len(listw1d5)
print(total)
print("# failed:")
print(count)

Total #: 
25
# failed:
0


In [32]:
#prints the ones that don't work for d = 5, w = 2
M = 5
count = 0
for l in listw2d5:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.08
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        count+=1
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")
print("Total #: ")
total = len(listw2d5)
print(total)
print("# failed:")
print(total - count)

Error Grid:
1 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
1 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
---------------
Error Grid:
1 0 1 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
1 0 1 0 0 
---------------
Error Grid:
1 0 0 1 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
1 0 0 1 0 
---------------
Error Grid:
1 0 0 0 1 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
1 0 0 0 1 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
---------------
Error Grid:
1 0 0 0 0 
1 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 1 0 0 
0 0 1 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
---------------
Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
---------------
Error Grid:
1 0 0 0 0 
0 0 1 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's 

In [33]:
#prints the ones that don't work for d = 5, w = 3
M = 5
count = 0
for l in listw3d5:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.12
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if not solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        count+=1
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")
print("Total #: ")
total = len(listw3d5)
print(total)
print("# failed:")
print(count)

Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 1 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 1 0 
0 0 0 0 1 
---------------
Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 1 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 1 
0 0 1 0 0 
---------------
Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 0 1 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 1 0 0 
0 0 0 1 0 
---------------
Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 1 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 1 
0 0 0 0 0 
0 0 0 1 0 
---------------
Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 1 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 1 
0 0 0 0 0 
0 0 1 0 0 
---------------
Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 0 1 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 0 0 0 0 
0 0 1 0 0 
0 0 0 0 0 
0 0 0 1 0 
---------------
Error Grid:
1 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 1 0 0 
Solver's 

In [35]:
#prints the ones that do work for d = 5, w = 3
M = 5
count = 0
for l in listw3d5:
    grid = create_grid(M)# no error
    add_y_error(grid,l)
    I= construct_stabilizers_scipy_I(M)
    C = construct_stabilizers_scipy_C(I, grid)
    p=0.12
    guess = print_result(*mle_decoder_bs(M**2,I,C,p))
    if solver_accuracy(M,grid,solver_to_grid_scipy(M,guess)):
        count+=1
        print("Error Grid:")
        Print(grid)
        print("Solver's guess:")
        Print(solver_to_grid_scipy(M,guess))
        print("---------------")
print("Total #: ")
total = len(listw3d5)
print(total)
print("# failed:")
print(total -count)

Error Grid:
1 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
1 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
---------------
Error Grid:
1 1 0 1 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
1 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 1 0 1 0 
---------------
Error Grid:
1 1 0 0 1 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
1 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 1 0 0 1 
0 0 0 0 0 
---------------
Error Grid:
1 1 0 0 0 
1 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
---------------
Error Grid:
1 1 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
1 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
---------------
Error Grid:
1 1 0 0 0 
0 0 1 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's guess:
0 0 0 0 0 
0 1 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
1 0 1 0 0 
---------------
Error Grid:
1 1 0 0 0 
0 0 0 1 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
Solver's 